# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

print("\n################################################################################\n")

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Not connected to a GPU

################################################################################

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


## GitHub

In [3]:
!ls
!git clone https://github.com/YunZhi246/subpopulation-data-poisoning-attacks.git

drive  sample_data
Cloning into 'subpopulation-data-poisoning-attacks'...
remote: Enumerating objects: 137, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 137 (delta 77), reused 98 (delta 41), pack-reused 0
Receiving objects: 100% (137/137), 1.61 MiB | 4.40 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [4]:
%cd /content/subpopulation-data-poisoning-attacks

/content/subpopulation-data-poisoning-attacks


In [5]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.9 MB 6.6 MB/s 
     |████████████████████████████████| 7.0 MB 25.5 MB/s 
     |████████████████████████████████| 26.0 MB 1.4 MB/s 
     |████████████████████████████████| 215 kB 55.3 MB/s 
     |████████████████████████████████| 4.4 MB 44.2 MB/s 
     |████████████████████████████████| 4.3 MB 54.1 MB/s 
     |████████████████████████████████| 451 kB 35.8 MB/s 
     |████████████████████████████████| 182 kB 76.0 MB/s 
     |████████████████████████████████| 6.6 MB 45.8 MB/s 
     |████████████████████████████████| 212 kB 76.3 MB/s 
     |████████████████████████████████| 132 kB 71.6 MB/s 
     |████████████████████████████████| 127 kB 59.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: scipy
    Found existin

# Results for BERT models

This notebook can be used to visualize the results of subpopulation attacks against BERT models, on the IMDB movie reviews dataset.

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
import os
import sys

In [ ]:
# os.chdir('../')
# os.environ['ML_DATA'] = ''  # Unused

In [8]:
import torch
import numpy as np
import pandas as pd

from torch.utils.data import TensorDataset, DataLoader
from transformers import BertForSequenceClassification

In [9]:
from attack_nlp import init_cluster_attack

from subclass_avail import common
from subclass_avail.target_nlp import bert_utils

## Constants

In [10]:
results_dir = '/content/drive/MyDrive/storage/results/xlnet'
fname = 'eval-stats_clus{}_pois{}_{}.npy'

n_clus = 100
seed = 42

pois_rates = ['0.5', '1.0', '2.0']
# m_types = ['LL', 'FT']
m_types = ['FT']

In [11]:
# Set the random seed to the same used during the attack
device = bert_utils.get_device()
bert_utils.set_seed(device=device, seed=seed)

Available device:  cpu


## Compare results


Let's first look at the subpopulation with highest target damage.
We will then look at the highest collateral damages.

In [16]:
# Accumulate all results in a single DataFrame
res_df = pd.DataFrame(columns=['type', 'p_rate', 'index', 't_dmg', 'p_acc', 'base_def', 'coll_dmg', 'csize', 'exp'], dtype=object)

for ps in pois_rates:
    for t in m_types:
        exp_name = fname.format(n_clus, ps, t)
        print('Experiment {}\n'.format(exp_name))

        res_arr = np.load(os.path.join(common.results_dir_xlnet, exp_name), allow_pickle=True).item()
        
        for clus_id, results in res_arr.items():
            if len(results['train_clus_size']) > 1:
                train_clus_size = len(results['train_clus_size'])
            else:
                train_clus_size = results['train_clus_size'][0]
            
            to_add = {
                'type': t,
                'p_rate': ps,
                'index': clus_id,
                't_dmg': results['base_def'] - results['pois'],
                'p_acc': results['pois'],
                'base_def': results['base_def'],
                'coll_dmg': results['collateral_dmg'],
                'csize': train_clus_size,
                'exp': exp_name
            }
            
            res_df = res_df.append(to_add, ignore_index=True)

Experiment eval-stats_clus100_pois0.5_FT.npy

Experiment eval-stats_clus100_pois1.0_FT.npy

Experiment eval-stats_clus100_pois2.0_FT.npy



In [17]:
# Sorting by target damage
for ps in pois_rates:
    for t in m_types:
        exp_name = fname.format(n_clus, ps, t)
        print('Experiment {}\n'.format(exp_name))
        
        sub_df = res_df[res_df['exp'] == exp_name]
        sub_df = sub_df.sort_values(by='t_dmg')
        
        top5_df = sub_df.tail(5)
        top10_df = sub_df.tail(10)
        
        print('Best target damage:')
        print(sub_df[-1:])
        print()
        
        print('Top 5 target damage averages:')
        print(top5_df.mean())
        print()

        print('Top 10 target damage averages:')
        print(top10_df.mean())
        print()
        
        print('-'*80)
        print()


Experiment eval-stats_clus100_pois0.5_FT.npy

Best target damage:
  type p_rate index     t_dmg     p_acc  base_def  coll_dmg csize  \
9   FT    0.5    47  0.070968  0.722581  0.793548  0.000845    75   

                                 exp  
9  eval-stats_clus100_pois0.5_FT.npy  

Top 5 target damage averages:
index        56.000000
t_dmg         0.050625
p_acc         0.768531
base_def      0.819156
coll_dmg      0.000883
csize       145.200000
dtype: float64

Top 10 target damage averages:
index        53.300000
t_dmg         0.037275
p_acc         0.791796
base_def      0.829071
coll_dmg      0.001007
csize       127.400000
dtype: float64

--------------------------------------------------------------------------------

Experiment eval-stats_clus100_pois1.0_FT.npy

Best target damage:
   type p_rate index     t_dmg     p_acc  base_def  coll_dmg csize  \
46   FT    1.0    83  0.097902  0.874126  0.972028  0.002253    74   

                                  exp  
46  eval-stats_clu

In [18]:
# Sorting by collateral damage
for ps in pois_rates:
    for t in m_types:
        exp_name = fname.format(n_clus, ps, t)
        print('Experiment {}\n'.format(exp_name))
        
        sub_df = res_df[res_df['exp'] == exp_name]
        sub_df = sub_df.sort_values(by='coll_dmg')
        
        print('Worst collateral damage:')
        print(sub_df[-1:])
        print(sub_df[-1:]['coll_dmg'] * 100)
        print()
        
        print('-'*80)
        print()


Experiment eval-stats_clus100_pois0.5_FT.npy

Worst collateral damage:
   type p_rate index  t_dmg     p_acc  base_def  coll_dmg csize  \
27   FT    0.5    56    0.0  0.996226  0.996226  0.003032   121   

                                  exp  
27  eval-stats_clus100_pois0.5_FT.npy  
27    0.303214
Name: coll_dmg, dtype: float64

--------------------------------------------------------------------------------

Experiment eval-stats_clus100_pois1.0_FT.npy

Worst collateral damage:
   type p_rate index     t_dmg     p_acc  base_def  coll_dmg csize  \
48   FT    1.0    54  0.022346  0.960894   0.98324  0.006452   180   

                                  exp  
48  eval-stats_clus100_pois1.0_FT.npy  
48    0.64524
Name: coll_dmg, dtype: float64

--------------------------------------------------------------------------------

Experiment eval-stats_clus100_pois2.0_FT.npy

Worst collateral damage:
   type p_rate index     t_dmg     p_acc  base_def  coll_dmg csize  \
82   FT    2.0    98  0.